In [1]:
import numpy as np
import pandas as pd
import os
import zipfile
import matplotlib.pyplot as plt
import glob
from config.config import *
from preprocessing.preprocessors import *
from analysis.analysis_functions import *
import re

no crisis measure selected.


In [2]:
os.chdir('C:\\Users\\Andy\\PycharmProjects\\finrlpaper1\\Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020\\')
cwd_ = os.getcwd() # get current working directory
cwd_

'C:\\Users\\Andy\\PycharmProjects\\finrlpaper1\\Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020'

### SETTINGS

In [3]:
class a_settings: # settings class only for analysis of results
    ### get all data saved from one run
    episodes_endings = [126, 189, 252, 315, 378, 441, 504, 567, 630, 693, 756, 819, 882, 945, 1008, 1071, 1134, 1197]

class a_paths: 
    results_dir = "results"
    datapath = "data/preprocessed/done_data.csv"
    indexpath = "data/raw/DJI.csv"
    # path where analysis results are saved
    save_path = os.path.join("analysis", "results")
    file_extension = '.csv'

class a_temp:
    #run_of_choice = "04-10-2021_03-22-17_ppo__fm2"
    run_of_choice = "04-16-2021_09-36-11_ppo__fm3"
    strategy_mode = "ppo"
    sample = "trade"
    n_seeds = 5

In [4]:
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

### Import preprocessed data & raw index data (DJIA 30)

In [5]:
data = load_dataset(file_path=a_paths.datapath,
                    col_subset=["datadate", "tic", "adjcp"])
data.head(3)

,datadate,tic,adjcp
0,20090102,AAPL,12.96
1,20090102,AXP,19.33
2,20090102,BA,45.25


In [6]:
dow30 = load_dataset(file_path=a_paths.indexpath,
                    col_subset=None) 
dow30.reset_index(inplace=True)
dow30.rename(columns={"Adj Close": "adjcp", "Date":"datadate"}, inplace=True)
dow30["datadate"] = pd.to_datetime(dow30["datadate"])
dow30 = dow30[["datadate", "adjcp"]]
dow30.head(3)

,datadate,adjcp
0,2009-01-02,9034.69
1,2009-01-05,8952.89
2,2009-01-06,9015.10


In [7]:
all_data_dict = {"data": data, "dow30": dow30}

In [8]:
data.datadate.dtype

dtype('int64')

In [9]:
dow30.datadate.dtype

dtype('<M8[ns]')

#### Get dates, tickers and episodes end iteration numbers

In [10]:
# get tickers
tickers = data.tic.unique()
print("tickers: ", tickers)
# get trades dates for validation
unique_trade_dates_validation = pd.read_csv(os.path.join(a_paths.results_dir,
                                                         "04-15-2021_23-12-43_ppo__fm3",
                                                         "unique_trade_dates_validation.csv"))#, index_col=0)
unique_trade_dates_validation = np.array(unique_trade_dates_validation)
print("\nvalidation dates: ")
unique_trade_dates_validation

tickers:  ['AAPL' 'AXP' 'BA' 'CAT' 'CSCO' 'CVX' 'DD' 'DIS' 'GS' 'HD' 'IBM' 'INTC'
 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PFE' 'PG' 'RTX' 'TRV'
 'UNH' 'V' 'VZ' 'WBA' 'WMT' 'XOM']

validation dates: 


array([[20151002],
       [20151005],
       [20151006],
       ...,
       [20200702],
       [20200706],
       [20200707]], dtype=int64)

In [11]:
#unique_trade_dates_validation = data[(data.datadate > settings.STARTDATE_VALIDATION) &
#                                    (data.datadate <= settings.ENDDATE_VALIDATION)].datadate.unique()
#unique_trade_dates_validation

In [12]:
trade_start = unique_trade_dates_validation[126-63][0]
trade_end = unique_trade_dates_validation[1197][0]
print(f"trade start: {trade_start}, trade end: {trade_end}")

trade start: 20160104, trade end: 20200707


In [13]:
trade_dates = data[(data.datadate>=trade_start)&(data.datadate<=trade_end)].datadate.unique()
trade_dates

array([20160104, 20160105, 20160106, ..., 20200702, 20200706, 20200707],
      dtype=int64)

### get subset of data and dow30 based on trade dates

In [14]:
data_tradedates = data[(data.datadate>=trade_start)&(data.datadate<=trade_end)]
data_tradedates["datadate"] =  pd.to_datetime(data_tradedates["datadate"], format='%Y%m%d')
data_tradedates.set_index('datadate', inplace=True)
data_tradedates

C:\Users\Andy\PycharmProjects\finrlpaper1\Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020\finrlpaperv4\Lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,tic,adjcp
datadate,,
2016-01-04,AAPL,105.35
2016-01-04,AXP,67.59
2016-01-04,BA,140.50
2016-01-04,CAT,67.99
2016-01-04,CSCO,26.41
...,...,...
2020-07-07,V,194.20
2020-07-07,VZ,55.04
2020-07-07,WBA,42.22


In [15]:
dow30_tradedates = dow30[(dow30.datadate>="2016-01-04")&(dow30.datadate<="2020-07-07")]
#dow30_tradedates["datadate"] =  pd.to_datetime(dow30_tradedates["datadate"], format='%Y%m%d')
dow30_tradedates.set_index('datadate', inplace=True)
dow30_tradedates

,adjcp
datadate,
2016-01-04,17148.94
2016-01-05,17158.66
2016-01-06,16906.51
2016-01-07,16514.10
2016-01-08,16346.45
...,...
2020-06-30,25812.88
2020-07-01,25734.97
2020-07-02,25827.36


In [16]:
all_data_dict.update({"data_tradedates": data_tradedates, "dow30_tradedates": dow30_tradedates})

In [17]:
list(all_data_dict.keys())

['data', 'dow30', 'data_tradedates', 'dow30_tradedates']

### get data for one run

In [18]:
all_seed_folders = get_all_fs(f_path=os.path.join(a_paths.results_dir, a_temp.run_of_choice), 
                              string_condition="*Seed*")
run_ppo__fm2_dict = create_run_dict(all_seed_folder_paths=all_seed_folders, 
                                    seed_string_condition=f"agent(.*)")

In [19]:
run_ppo__fm2_dict["Seed80923"].keys()

dict_keys(['buy_trades', 'cash_value', 'datadates', 'exercised_actions', 'last_state', 'number_asset_holdings', 'policy_actions', 'portfolio_value', 'rewards', 'sell_trades', 'state_memory', 'transaction_cost'])

In [20]:
print(run_ppo__fm2_dict.keys())
print("")
print(run_ppo__fm2_dict["Seed11112"].keys())
print("")
print(run_ppo__fm2_dict["Seed11112"]["buy_trades"].keys())
print("")
print(run_ppo__fm2_dict["Seed11112"]["buy_trades"]["train"][0:2])

dict_keys(['Seed11112', 'Seed223445', 'Seed23', 'Seed80923'])

dict_keys(['buy_trades', 'cash_value', 'datadates', 'exercised_actions', 'last_state', 'number_asset_holdings', 'policy_actions', 'portfolio_value', 'rewards', 'sell_trades', 'state_memory', 'transaction_cost'])

dict_keys(['train', 'validation', 'trade'])

['results\\04-16-2021_09-36-11_ppo__fm3\\agentSeed11112\\buy_trades\\buy_trades_train_ppo_i1008.csv', 'results\\04-16-2021_09-36-11_ppo__fm3\\agentSeed11112\\buy_trades\\buy_trades_train_ppo_i1071.csv']


In [21]:
# results to be loaded as pandas df based on the path in the dict
load_list = ["portfolio_value", "cash_value", "rewards", "policy_actions", 
             "exercised_actions", "number_asset_holdings", "transaction_cost"]
dicty, fp = load_combine_results(dictionary=run_ppo__fm2_dict.copy(), load_list=load_list, 
                                 mode="trade", combine_mode="within_seed", 
                                 sorting=True, sorting_condition="_i(.*).csv")
#dicty

In [ ]:

dicty, fp = load_combine_results(dictionary=run_ppo__fm2_dict.copy(), load_list=load_list, 
                                 mode="trade", combine_mode="within_seed", 
                                 sorting=True, sorting_condition="_i(.*).csv")

In [58]:
from functools import reduce
dict_ = dicty.copy()
dic = {}
mode = "trade"
for result_elem in load_list:
    #print(result_elem)
    temp_list = []
    for seed_key in dict_:
        #print(seed_key)
        seed_df = dict_[seed_key][result_elem][mode]
        seed_df.rename(columns={result_elem: result_elem+"_"+seed_key}, inplace=True)
        #seed_df.set_index("datadate", inplace=True)
        temp_list.append(seed_df)
    #df_conc = pd.concat(temp_list, axis=1, ignore_index=True)
    df_conc = reduce(lambda left, right: pd.merge(left, right, on = 'datadate'), temp_list)
    dic.update({result_elem: df_conc})

portfolio_value
Seed11112
Seed223445
Seed23
Seed80923
cash_value
Seed11112
Seed223445
Seed23
Seed80923
rewards
Seed11112
Seed223445
Seed23
Seed80923
policy_actions
Seed11112
Seed223445
Seed23
Seed80923
exercised_actions
Seed11112
Seed223445
Seed23
Seed80923
number_asset_holdings
Seed11112
Seed223445
Seed23
Seed80923
transaction_cost
Seed11112
Seed223445
Seed23
Seed80923


In [60]:
dic
#temp_list

{'portfolio_value':      datadate  portfolio_value_Seed11112  portfolio_value_Seed223445  \
 0    20160104                 1000000.00                  1000000.00   
 1    20160105                 1000253.97                   999681.27   
 2    20160106                  998334.47                   998010.52   
 3    20160107                  995372.86                   993550.40   
 4    20160108                  993086.46                   991137.53   
 ..        ...                        ...                         ...   
 121  20160627                 1071815.32                  1046022.08   
 122  20160628                 1081951.43                  1060083.74   
 123  20160629                 1101309.47                  1081585.76   
 124  20160630                 1115901.48                  1094518.42   
 125  20160701                 1116149.45                  1094005.76   
 
      portfolio_value_Seed23  portfolio_value_Seed80923  
 0                1000000.00                 

In [61]:
dic["transaction_cost"]

,datadate,transaction_cost_Seed11112,transaction_cost_Seed223445,transaction_cost_Seed23,transaction_cost_Seed80923
0,20160104,63.53,89.32,133.55,55.80
1,20160105,125.44,111.36,140.84,93.14
2,20160106,114.83,126.48,126.14,93.63
3,20160107,150.37,129.32,155.48,97.29
4,20160108,135.29,131.07,146.22,126.40
...,...,...,...,...,...
121,20160627,56.61,46.66,47.90,45.81
122,20160628,66.24,62.68,24.65,91.69
123,20160629,80.51,103.79,30.64,67.25
124,20160630,20.31,71.82,0.96,112.94


In [22]:
dicty2, _ = load_combine_results(dictionary=dicty.copy(), load_list=load_list, 
                                 mode="trade", combine_mode="across_seeds",
                                 sorting=False, sorting_condition="_i(.*).csv")
dicty2

portfolio_value
Seed11112
Seed223445
Seed80923
cash_value
Seed11112
Seed223445
Seed80923
portfolio_value


KeyError: 'cash_value'

In [24]:
dicty2

NameError: name 'dicty2' is not defined

In [ ]:
# https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/
# https://stackoverflow.com/questions/6633678/finding-words-after-keyword-in-python
# fill seed dicts with portfolio values dfs for each iteration


def prepare_results(seedrun_dirnames_list=[dirname1, dirname2, dirname3, dirname4, dirname5], 
                    dictionary=portfolio_value_dicts,#={"seed5": seed5_dict, 
                                               #seed23": seed23_dict,
                                               #"seed11112": seed11112_dict, 
                                              # "seed80923": seed80923_dict, 
                                               #"seed223445": seed223445_dict},
                    seednames=seedrun_names,
                    targetname="end_portfolio_value_nextDayOpening_", 
                    sample="trade", 
                    strategy_mode="ppo", 
                    episodes_endings=[126, 189, 252, 315, 378, 441, 504, 567, 630, 
                                   693, 756, 819, 882, 945, 1008, 1071, 1134, 1197], 
                    extension='.csv',
                    n_seeds=5, 
                    filesformat="1Col",
                    multicol_names=tickers,
                    exercised_policy=False,
                    pf_value=True):
    
    ##### get results from directory and put them into dict as a list of dfs
    #####-------------------------------------------------------------------
    run = 1 
    filenames_list = []   
    #combine all filenames in a list
    colnames = []
    
    for (seed_dirname, key) in zip(seedrun_dirnames_list, portfolio_value_dicts.keys()):
        #print(key)
        #print(seed_dirname

        # get all filenames for portfolio value for the different episodes
        subdir_path = os.path.join(seed_dirname, subdir_target)
        #all_filenames = glob.glob(os.path.join(subdir_path, "*{}_{}_i*.{}".format(sample, strategy_mode, extension)))

        iteration_files_list = []

        if run==1: # do it only once since always the same names for each seed
            for episode in episodes_endings:
                main_filename = targetname+sample+"_"+strategy_mode+"_i"+str(episode)
                #print(main_filename)
                filenames_list.append(main_filename)

            #print(filenames_list)

        for filename in filenames_list:

            # get shortname for iteration
            shortname = re.search('(?<={}_)(.*)'.format(re_starting_point), filename)
            shortname = shortname.group(1)# + shortname.group(2)
            if run==1: colnames.append(shortname) #are same for each seed

            # get file and save to dict with shortname for iteration
            df = pd.read_csv(os.path.join(subdir_path,filename+extension), index_col=0)
            
            if run == 1 and exercised_policy==True: 
                df
                #print(df) # for debug manually
                #print("df shape; ", len(df))
                
            if filesformat=="MultiCol" and exercised_policy==False: 
                # if we have multiple columns, we name them based on multicol_names
                mystring = str(key)+"_"
                df.columns = [mystring + s for s in multicol_names]
            
            iteration_files_list.append(df)

        dictionary[key].update({sample: iteration_files_list})

        run+=1
        
    dicttemp = dictionary.copy()
        
    ##### combine data to df; 
    ##### first all iterations in one times series, then all seed time series in one df
    #####-------------------------------------------------------------------------------
    df_list2 = []
    for key in list(dictionary.keys())[:n_seeds]:
        #print(key)

        #print(dictionary[key])
        df_list1 = dictionary[key][sample] # the list of all time series dfs for each iteration (18 in total),
                                            # each of different length
        # delete first entry in all but the first df (because it is also the endig of the previous df, hence double) 
        #print(df_list1)
        if pf_value==True:
            e = 1

            for elem in df_list1[1:]:
                #print(elem)
                tempdf = elem
                tempdf = tempdf.iloc[1:]

                #if filesformat=="MultiCol": 
                    # if we have multiple columns, we name them based on multicol_names
                    #mystring = str(key)+"_"
                    #tempdf.columns = [mystring + s for s in multicol_names]

                df_list1[e] = tempdf
                #print(tempdf)
                e+=1

        df = pd.concat(df_list1, axis=0).reset_index(drop=True) # combine all elements in list to one df, with one 
        # or multiple columns (in case of actions or number of holdings)
        #df.columns = colnames[i]
        dictionary[key][sample] = df # insert df into where the list was previously
        #print(df)
        # append this df to another list
        df_list2.append(df)
    # combine all elements in df_list2 to one df again, but this time to multiple cols (one for each seed)
    # and save under a newly generated key in the dictionary
    
    df = pd.concat(df_list2, axis=1)
    dictionary.update({"all_seeds_"+sample : df})
    if filesformat=="1Col": dictionary["all_seeds_"+sample].columns = seednames


    return [dictionary, colnames, dicttemp]

In [ ]:
# all directories names for this run
dirname1 = "04-10-2021_03-22-17_agentSeed5_ppo__fm2"
dirname2 = "04-10-2021_03-22-17_agentSeed23_ppo__fm2"
dirname3 = "04-10-2021_03-22-17_agentSeed11112_ppo__fm2"
dirname4 = "04-10-2021_03-22-17_agentSeed80923_ppo__fm2"
dirname5 = "04-10-2021_03-22-17_agentSeed223445_ppo__fm2"
# create a list for the directories names
seedrun_dirnames_list = [dirname1, dirname2, dirname3, dirname4, dirname5]
seedrun_names = ["seed5", "seed23", "seed11112", "seed80923", "seed223445"]
#seed5_dict, seed23_dict, seed11112_dict, seed80923_dict, seed223445_dict = {},{},{},{},{}
#seed_dict_names = ["seed5_dict", "seed23_dict", "seed11112_dict", "seed80923_dict", "seed223445_dict"]

##### Functions

In [ ]:
extension = '.csv'
sample = "trade"
strategy_mode = "ppo"
n_seeds = 5
# summarizing dict, will be changed later
#DICT_PPO_NOTURB_F1_32217 = paths.SUBDIR_NAMES
#DICT_PPO_NOTURB_F1_32217

##### Functions

In [ ]:
def makeplot(dataset,
             results_data,
             #x_colnames="datadate", 
             y_colnames_list=seedrun_names,
             title1='Portfolio Value trading period from ',
             title2='Stock Price trading period from',
             startdate=20160101,
             enddate=20200817,
             ylabel1="Portfolio Value incl. Cash",
             ylabel2="Stock Price",
             xlabel="Date",
             n_plots=2,
             figsize_l=20,
             figsize_r=20):

    if n_plots == 2:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(figsize_l, figsize_r))
        # make a little extra space between the subplots
        fig.subplots_adjust(hspace=0.5)

        for colname in y_colnames_list:
            #ax1.plot(results_data[x_colnames], results_data[colname], label=colname)
            ax1.plot(results_data.index, results_data[colname], label=colname)

        #ax1.ticklabel_format(style='plain')
        ax1.set_title(title1+" "+str(startdate)+" until "+str(enddate)+".")
        ax1.set_xlabel(xlabel, fontsize=16)
        ax1.set_ylabel(ylabel1, fontsize=16)
        #ax1.set_ylim(bottom=0, top=15000)
        ax1.legend(loc="best")

        for ticker in tickers:
            ax2.plot(dataset[dataset.tic==ticker].adjcp, label=ticker)
        #ax2.ticklabel_format(style='plain')
        ax2.set_title(title2+"_"+str(startdate)+" until "+str(enddate)+".")
        ax2.set_xlabel(xlabel, fontsize=16)
        ax2.set_ylabel(ylabel2, fontsize=16)
        #ax2.set_ylim(bottom=0, top=15000)
        ax2.legend(loc="best")
        
    elif n_plots == 1:
        fig, ax1 = plt.subplots(1, 1, figsize=(figsize_l, figsize_r))
        # make a little extra space between the subplots
        fig.subplots_adjust(hspace=0.5)

        for colname in y_colnames_list:
            #ax1.plot(results_data[x_colnames], results_data[colname], label=colname)
            ax1.plot(results_data.index, results_data[colname], label=colname)

        #ax1.ticklabel_format(style='plain')
        ax1.set_title(title1+" "+str(startdate)+" until "+str(enddate)+".")
        ax1.set_xlabel(xlabel, fontsize=16)
        ax1.set_ylabel(ylabel1, fontsize=16)
        #ax1.set_ylim(bottom=0, top=15000)
        ax1.legend(loc="best")
        
    elif n_plots == "twin":
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(figsize_l, figsize_r))
        # make a little extra space between the subplots
        fig.subplots_adjust(hspace=0.5)

        for colname in y_colnames_list:
            #ax1.plot(results_data[x_colnames], results_data[colname], label=colname)
            ax1.plot(results_data.index, results_data[colname], label=colname)

        #ax1.ticklabel_format(style='plain')
        ax1.set_title(title1+" "+str(startdate)+" until "+str(enddate)+".")
        ax1.set_xlabel(xlabel, fontsize=16)
        ax1.set_ylabel(ylabel1, fontsize=16)
        #ax1.set_ylim(bottom=0, top=15000)
        ax1.legend(loc="best")

        for colname in y_colnames_list:
            ax2.plot(dataset.index, dataset[colname], label=colname)

        #ax1.ticklabel_format(style='plain')
        ax2.set_title(title2+" "+str(startdate)+" until "+str(enddate)+".")
        ax2.set_xlabel(xlabel, fontsize=16)
        ax2.set_ylabel(ylabel2, fontsize=16)
        #ax2.set_ylim(bottom=0, top=15000)
        ax2.legend(loc="best")    
        
    elif n_plots == "results_and_index":
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(figsize_l, figsize_r))
        # make a little extra space between the subplots
        fig.subplots_adjust(hspace=0.5)

        for colname in y_colnames_list:
            #ax1.plot(results_data[x_colnames], results_data[colname], label=colname)
            ax1.plot(results_data.index, results_data[colname], label=colname)

        #ax1.ticklabel_format(style='plain')
        ax1.set_title(title1+" "+str(startdate)+" until "+str(enddate)+".")
        ax1.set_xlabel(xlabel, fontsize=16)
        ax1.set_ylabel(ylabel1, fontsize=16)
        #ax1.set_ylim(bottom=0, top=15000)
        ax1.legend(loc="best")

        ax2.plot(dataset.index, dataset.adjcp, label="adjcp")
        #ax1.ticklabel_format(style='plain')
        ax2.set_title(title2+" "+str(startdate)+" until "+str(enddate)+".")
        ax2.set_xlabel(xlabel, fontsize=16)
        ax2.set_ylabel(ylabel2, fontsize=16)
        #ax2.set_ylim(bottom=0, top=15000)
        ax2.legend(loc="best")    

### Portfolio value

In [ ]:
subdir_target = "portfolio_value"
re_starting_point = "Opening"

portfolio_value_dicts = {"seed5": {}, "seed23": {},
                         "seed11112": {}, "seed80923": {}, 
                         "seed223445": {}}

targetname = "end_portfolio_value_nextDayOpening_"

# will be: 
# portfolio_value_dicts = {"seed5": seed5_dict = {"trade": list(iterations csvs), "train: ,..."}, 
#                          "seed23": seed23_dict = {"trade": list(iterations csvs), "train: ,..."}}

#### get results data

In [ ]:
#portfolio_value_dicts["seed5"]["trade"]
portfolio_value_dict_, colnames, dt = prepare_results(seedrun_dirnames_list=seedrun_dirnames_list, 
                                                      dictionary=portfolio_value_dicts,
                                                      seednames=seedrun_names,
                                                      targetname="end_portfolio_value_nextDayOpening_", 
                                                      sample="trade", 
                                                      strategy_mode="ppo", 
                                                      episodes_ends=[126, 189, 252, 315, 378, 441, 504, 567, 
                                                                     630, 693, 756, 819, 882, 945, 1008, 
                                                                     1071, 1134, 1197], 
                                                      extension='.csv',
                                                      n_seeds=5, 
                                                      filesformat="1Col",
                                                      multicol_names=tickers,
                                                      exercised_policy=False,
                                                      pf_value=True)
#print(pfval_dict)
#print("")
#print(colnames)

In [ ]:
#colnames
#dt
#portfolio_value_dict_["seed5"]["trade"]#.iloc[0:20]

In [ ]:
#portfolio_value_dict_["all_seeds_trade"]

In [ ]:
# we know; row 64 is the last time point of the validation dataset, 
# which ends at:  20200707 (settings.ENDDATE_VALIDATION)
# iteration 1197 = date 20200707

#### combine results with dates

In [ ]:
results_data = portfolio_value_dict_["all_seeds_trade"]
# combine trade dates with our results
results_data["datadate"] = trade_dates
results_data["datadate"] =  pd.to_datetime(results_data["datadate"], format='%Y%m%d')

results_data.set_index('datadate', inplace=True)
results_data

In [ ]:
results_data['seed223445']

#### plot results

In [ ]:
makeplot(dataset=dow30_tradedates, 
         results_data=results_data,
         y_colnames_list=seedrun_names,
         title1='Portfolio Value trading period from ',
         title2='Index Value trading period from',
         startdate=20160101,
         enddate=20200817,
         ylabel1="Portfolio Value incl. Cash",
         ylabel2="Index Value",
         xlabel="Date",
         n_plots="results_and_index",
         figsize_l=10,
         figsize_r=10)

### Cash value

In [ ]:
subdir_target = "cash_value"
re_starting_point = "value"

cash_value_dicts = {"seed5": {}, "seed23": {},
                         "seed11112": {}, "seed80923": {}, 
                         "seed223445": {}}

targetname = "cash_value_"

In [ ]:
cash_value_dicts

In [ ]:
cash_value_dict_, _, _ = prepare_results(seedrun_dirnames_list=[dirname1, dirname2, dirname3, dirname4, dirname5], 
                        dictionary=cash_value_dicts,#={"seed5": seed5_dict, 
                                               #seed23": seed23_dict,
                                               #"seed11112": seed11112_dict, 
                                              # "seed80923": seed80923_dict, 
                                               #"seed223445": seed223445_dict},
                        seednames=seedrun_names,
                        targetname=targetname, 
                        sample="trade", 
                        strategy_mode="ppo", 
                        episodes_ends=[126, 189, 252, 315, 378, 441, 504, 567, 630, 
                                       693, 756, 819, 882, 945, 1008, 1071, 1134, 1197], 
                        extension='.csv',
                           n_seeds=5)
# output = [dictionary, colnames, dicttemp]

In [ ]:
#cash_value_dict_

In [ ]:
cash_data = cash_value_dict_["all_seeds_trade"]
# combine trade dates with our results
cash_data["datadate"] = trade_dates
cash_data["datadate"] =  pd.to_datetime(cash_data["datadate"], format='%Y%m%d')

cash_data.set_index('datadate', inplace=True)
cash_data

In [ ]:
#makeplot(dataset=results_data, 
#         results_data=cash_data,
#         y_colnames_list=seedrun_names,
#         title1='Cash Value trading period from ',
#         title2='Stock Price trading period from',
#         startdate=20160101,
#         enddate=20200817,
#         ylabel1="Cash Value",
#         ylabel2="Portfolio Value",
#         xlabel="Date",
#         n_plots=1)

### Subtraction:  Pf incl. Cash - Cash to get Asset-only value over time

In [ ]:
asset_only_data = results_data-cash_data
asset_only_data

In [ ]:
makeplot(dataset=dataset_tradedates, 
         results_data=asset_only_data,
         y_colnames_list=seedrun_names,
         title1='Portfolio Value (Asset Only) trading period from ',
         title2='Stock Price trading period from',
         startdate=20160101,
         enddate=20200817,
         ylabel1="Portfolio Value (Asset Only)",
         ylabel2="Stock Price",
         xlabel="Date",
         n_plots=2)

### Policy vs. exercised policy

In [ ]:
subdir_target = "policy_actions"
re_starting_point = "actions"
policyActions_dicts = {"seed5": {}, "seed23": {},
                         "seed11112": {}, "seed80923": {}, 
                         "seed223445": {}}

targetname_policyActions = "policy_actions_"

In [ ]:
policyActions_dict_, _, dt = prepare_results(seedrun_dirnames_list=seedrun_dirnames_list, 
                                            dictionary=policyActions_dicts,#={"seed5": seed5_dict, 
                                               #seed23": seed23_dict,
                                               #"seed11112": seed11112_dict, 
                                              # "seed80923": seed80923_dict, 
                                               #"seed223445": seed223445_dict},
                                            seednames=seedrun_names,
                                            targetname=targetname_policyActions, 
                                            sample="trade", 
                                            strategy_mode="ppo", 
                                            episodes_ends=[126, 189, 252, 315, 378, 441, 504, 567, 630, 
                                                           693, 756, 819, 882, 945, 1008, 1071, 1134, 1197], 
                                            extension='.csv',
                                            n_seeds=5,
                                            filesformat="MultiCol",
                                            multicol_names=tickers)
# output = [dictionary, colnames, dicttemp]

In [ ]:
#policyActions_dict_

In [ ]:
subdir_target = "exercised_actions"
re_starting_point = "actions"
ExercisedActions_dicts = {"seed5": {}, "seed23": {},
                         "seed11112": {}, "seed80923": {}, 
                         "seed223445": {}}

targetname_ExercisedActions = "exercised_actions_"

In [ ]:
ExercisedActions_dict_, _, dt = prepare_results(seedrun_dirnames_list=seedrun_dirnames_list, 
                                            dictionary=ExercisedActions_dicts,#={"seed5": seed5_dict, 
                                               #seed23": seed23_dict,
                                               #"seed11112": seed11112_dict, 
                                              # "seed80923": seed80923_dict, 
                                               #"seed223445": seed223445_dict},
                                            seednames=seedrun_names,
                                            targetname=targetname_ExercisedActions, 
                                            sample="trade", 
                                            strategy_mode="ppo", 
                                            episodes_ends=[126, 189, 252, 315, 378, 441, 504, 567, 630, 
                                                           693, 756, 819, 882, 945, 1008, 1071, 1134, 1197], 
                                            extension='.csv',
                                            n_seeds=5,
                                            filesformat="MultiCol",
                                            multicol_names=tickers)

In [ ]:
makeplot(dataset=dataset_tradedates, 
         results_data=asset_only_data,
         y_colnames_list=seedrun_names,
         title1='Portfolio Value (Asset Only) trading period from ',
         title2='Stock Price trading period from',
         startdate=20160101,
         enddate=20200817,
         ylabel1="Portfolio Value (Asset Only)",
         ylabel2="Stock Price",
         xlabel="Date",
         n_plots="twin")